In [ ]:
import os
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers

# Set the local weights file
local_weights_file = './inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

# Initialize the pre-trained model
pre_trained_model = InceptionV3(input_shape=(150, 150, 3),
                                include_top=False,
                                weights=None)

# Load the local weights
pre_trained_model.load_weights(local_weights_file)

# Freeze the layers
for layer in pre_trained_model.layers:
    layer.trainable = False

pre_trained_model.summary()

In [3]:
last_layer = pre_trained_model.get_layer('mixed7')
last_output = last_layer.output

In [ ]:
x = tf.keras.layers.Flatten()(last_output)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(pre_trained_model.input, x)
model.summary()

In [5]:
BASE_DIR = '/Users/aditya/Desktop/Kaggle/catdog/prepared_data'

train_dir = os.path.join(BASE_DIR, 'train')
validation_dir = os.path.join(BASE_DIR, 'validation')

# Directory with training cat/dog pictures
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with validation cat/dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

print(f"Contents of base directory: {os.listdir(BASE_DIR)}")

print(f"\nContents of train directory: {os.listdir(train_dir)}")

print(f"\nContents of validation directory: {os.listdir(validation_dir)}")

Contents of base directory: ['.DS_Store', 'train', 'validation']

Contents of train directory: ['dogs', 'cats', '.DS_Store']

Contents of validation directory: ['dogs', 'cats', '.DS_Store']


In [6]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size = (150, 150),
    batch_size = 20,
    label_mode = 'binary'
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    validation_dir,
    image_size = (150, 150),
    batch_size = 20,
    label_mode = 'binary'
)

Found 22546 files belonging to 2 classes.
Found 2454 files belonging to 2 classes.


In [7]:
def preprocess(image, label):
    image = tf.keras.applications.inception_v3.preprocess_input(image)
    return image, label

train_dataset_scaled = train_dataset.map(preprocess)
validation_dataset_scaled = validation_dataset.map(preprocess)

In [9]:
SHUFFLE_BUFFER_SIZE = 1000
PREFETCH_BUFFER_SIZE = tf.data.AUTOTUNE

train_dataset_final = (train_dataset_scaled
                        .cache()
                        .shuffle(SHUFFLE_BUFFER_SIZE)
                        .prefetch(PREFETCH_BUFFER_SIZE)
                    )

validation_dataset_final = (validation_dataset_scaled
                            .cache()
                            .prefetch(PREFETCH_BUFFER_SIZE))

In [12]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("Horizontal"),
    tf.keras.layers.RandomRotation(0.4),
    tf.keras.layers.RandomTranslation(0.2, 0.2),
    tf.keras.layers.RandomContrast(0.4),
    tf.keras.layers.RandomZoom(0.2)
])

In [13]:
inputs = tf.keras.Input(shape =(150,150,3))
x = data_augmentation(inputs)
x = model(x)

model_with_aug = tf.keras.Model(inputs, x)

In [14]:
model_with_aug.compile(
    optimizer = tf.keras.optimizers.RMSprop(learning_rate = 0.001),
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

In [15]:
history = model_with_aug.fit(
    train_dataset_final,
    epochs=20,
    validation_data=validation_dataset_final,
    verbose=2
)

Epoch 1/20


2026-01-30 10:11:38.839383: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1128/1128 - 104s - 92ms/step - accuracy: 0.6944 - loss: 1.4492 - val_accuracy: 0.8187 - val_loss: 0.7625
Epoch 2/20
1128/1128 - 93s - 83ms/step - accuracy: 0.7047 - loss: 0.9516 - val_accuracy: 0.9438 - val_loss: 0.1632
Epoch 3/20
1128/1128 - 92s - 81ms/step - accuracy: 0.7236 - loss: 0.8540 - val_accuracy: 0.9462 - val_loss: 0.1497
Epoch 4/20
1128/1128 - 90s - 80ms/step - accuracy: 0.7302 - loss: 0.8139 - val_accuracy: 0.9372 - val_loss: 0.1954
Epoch 5/20
1128/1128 - 92s - 82ms/step - accuracy: 0.7401 - loss: 0.7758 - val_accuracy: 0.9576 - val_loss: 0.1314
Epoch 6/20
1128/1128 - 92s - 82ms/step - accuracy: 0.7415 - loss: 0.7906 - val_accuracy: 0.9523 - val_loss: 0.1509
Epoch 7/20
1128/1128 - 92s - 81ms/step - accuracy: 0.7424 - loss: 0.7855 - val_accuracy: 0.9511 - val_loss: 0.1572
Epoch 8/20
1128/1128 - 93s - 83ms/step - accuracy: 0.7416 - loss: 0.7904 - val_accuracy: 0.9377 - val_loss: 0.2232
Epoch 9/20
1128/1128 - 92s - 82ms/step - accuracy: 0.7464 - loss: 0.7788 - val_accuracy: 0

In [16]:
def plot_loss_acc(history):
    '''Plots the training and validation loss and accuracy from a history object'''
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    epochs = range(len(acc))
    
    fig, ax = plt.subplots(1,2, figsize=(12, 6))
    ax[0].plot(epochs, acc, 'bo', label='Training accuracy')
    ax[0].plot(epochs, val_acc, 'b', label='Validation accuracy')
    ax[0].set_title('Training and validation accuracy')
    ax[0].set_xlabel('epochs')
    ax[0].set_ylabel('accuracy')
    ax[0].legend()
    
    ax[1].plot(epochs, loss, 'bo', label='Training Loss')
    ax[1].plot(epochs, val_loss, 'b', label='Validation Loss')
    ax[1].set_title('Training and validation loss')
    ax[1].set_xlabel('epochs')
    ax[1].set_ylabel('loss')
    ax[1].legend()
    
    plt.show()

plot_loss_acc(history)

NameError: name 'plt' is not defined